In [2]:
import pandas as pd
import glob

In [3]:
# 여러 날짜의 데이터 파일을 불러옵니다.
file_list = glob.glob('../dataset/Milano/raw/sms-call-internet-mi-2013-11-*.txt')

# 모든 데이터를 저장할 빈 리스트를 만듭니다.
all_data = []

# 각 파일을 순회하면서 데이터를 읽어옵니다.
for file in file_list:
    data = pd.read_csv(file, sep='\t', header=None)
    data.columns = ['CellID', 'Timestamp', 'CountryCode', 'smsin', 'smsout', 'callin', 'callout', 'internet']
    all_data.append(data)


print(all_data)

[         CellID      Timestamp  CountryCode     smsin    smsout    callin  \
0             1  1383260400000            0  0.081363       NaN       NaN   
1             1  1383260400000           39  0.141864  0.156787  0.160938   
2             1  1383261000000            0  0.136588       NaN       NaN   
3             1  1383261000000           33       NaN       NaN       NaN   
4             1  1383261000000           39  0.278452  0.119926  0.188777   
...         ...            ...          ...       ...       ...       ...   
4842620    9999  1383345600000            0  0.159189       NaN       NaN   
4842621    9999  1383345600000           33  0.014708       NaN       NaN   
4842622    9999  1383345600000           39  0.646086  0.550928       NaN   
4842623    9999  1383346200000           39  0.894907  0.518741  0.085995   
4842624    9999  1383346200000           49       NaN       NaN       NaN   

          callout   internet  
0             NaN        NaN  
1        0.0

In [7]:
all_data[5]

,CellID,Timestamp,CountryCode,smsin,smsout,callin,callout,internet
0,1,1383778800000,0,0.026137,NaN,NaN,NaN,NaN
1,1,1383778800000,39,0.110989,0.379342,0.110989,0.026137,8.709982
2,1,1383779400000,0,0.081901,NaN,NaN,NaN,NaN
3,1,1383779400000,39,0.247491,0.248116,NaN,NaN,9.273107
4,1,1383780000000,0,0.081901,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
5886878,9999,1383864000000,39,0.914131,1.428025,0.669794,NaN,17.813618
5886879,9999,1383864600000,0,0.092566,NaN,NaN,0.088767,NaN
5886880,9999,1383864600000,216,0.288963,NaN,NaN,NaN,NaN
5886881,9999,1383864600000,221,0.171990,NaN,NaN,0.085995,NaN


In [ ]:
# 모든 데이터를 하나의 데이터프레임으로 합칩니다.
df = pd.concat(all_data, ignore_index=True)

# Timestamp를 datetime 형식으로 변환합니다.
df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='ms')

In [4]:
# 1. CountryCode를 제거합니다.
df = df.drop(columns=['CountryCode'])

# 2. 각 그리드의 결측치 비율을 계산하고, 결측치가 가장 적은 그리드를 선택합니다.
# 각 그리드별로 결측치가 아닌 값의 개수를 세고 가장 많은 그리드를 선택합니다.
non_null_counts = df.groupby('CellID').count()
selected_grid = non_null_counts.idxmax().max()

# 선택된 그리드의 데이터만 필터링합니다.
df_selected_grid = df[df['CellID'] == selected_grid]
# 이제 같은 그리드이므로 CellID 삭제
df_selected_grid = df_selected_grid.drop(columns=['CellID'])

# 3. 같은 그리드의 시간대를 기준으로 데이터를 합산
# Timestamp를 10분 단위로 내림합니다.
df_selected_grid['Timestamp'] = df_selected_grid['Timestamp'].dt.floor('10T')

# 그룹화 및 합산
summed_cdrs = df_selected_grid.groupby('Timestamp').sum().reset_index()

summed_cdrs=summed_cdrs.rename(columns={'internet': 'OT','Timestamp':'date'})

/tmp/ipykernel_2841040/4091263985.py:16: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df_selected_grid['Timestamp'] = df_selected_grid['Timestamp'].dt.floor('10T')


In [5]:
summed_cdrs

,date,smsin,smsout,callin,callout,OT
0,2013-10-31 23:00:00,14.041196,10.849898,5.739816,7.912167,276.087402
1,2013-10-31 23:10:00,17.262998,10.234269,11.753134,7.848578,279.427423
2,2013-10-31 23:20:00,9.133549,3.759300,7.789001,4.465201,365.590192
3,2013-10-31 23:30:00,18.510871,9.546967,4.819154,7.026991,331.777771
4,2013-10-31 23:40:00,13.713547,5.581783,6.518861,3.491299,326.530946
...,...,...,...,...,...,...
4315,2013-11-30 22:10:00,20.242036,16.290376,9.008528,6.691303,448.892832
4316,2013-11-30 22:20:00,17.283562,13.319806,5.594521,9.745538,389.982287
4317,2013-11-30 22:30:00,22.847303,15.418740,13.817545,7.379469,387.052578
4318,2013-11-30 22:40:00,13.610373,12.999483,12.993111,15.157638,377.590362


In [6]:
# 결과 확인
print(f"선택된 그리드 ID: {selected_grid}")
print(summed_cdrs.head())

선택된 그리드 ID: 6165
                 date      smsin     smsout     callin   callout          OT
0 2013-10-31 23:00:00  14.041196  10.849898   5.739816  7.912167  276.087402
1 2013-10-31 23:10:00  17.262998  10.234269  11.753134  7.848578  279.427423
2 2013-10-31 23:20:00   9.133549   3.759300   7.789001  4.465201  365.590192
3 2013-10-31 23:30:00  18.510871   9.546967   4.819154  7.026991  331.777771
4 2013-10-31 23:40:00  13.713547   5.581783   6.518861  3.491299  326.530946


In [7]:
summed_cdrs.shape

(4320, 6)

In [8]:
# 필요한 경우 결과를 CSV 파일로 저장
summed_cdrs.to_csv('../dataset/Milano/temp/Milano_traffic.csv', index=False)